<a href="https://colab.research.google.com/github/Wezz-git/AI-samples/blob/main/Volatility_Forecasting_(GARCH_vs_LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.3/981.3 kB 16.8 MB/s eta 0:00:00


In [ ]:
from arch import arch_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error

In [ ]:
data = yf.download('AAPL', start='2010-01-01', end='2023-12-31')

/tmp/ipython-input-3748985763.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('AAPL', start='2010-01-01', end='2023-12-31')
[*********************100%***********************]  1 of 1 completed


In [ ]:
# Will need this to check the columns for datasets
print(data.columns)

MultiIndex([( 'Close', 'AAPL'),
            (  'High', 'AAPL'),
            (   'Low', 'AAPL'),
            (  'Open', 'AAPL'),
            ('Volume', 'AAPL')],
           names=['Price', 'Ticker'])


In [ ]:
df = data[['Close']].copy()

In [ ]:
df['Return'] = np.log(df['Close'] / df['Close'].shift(1))

#Drop the first NaN (Not a Number) value
df = df.dropna()

In [ ]:
#Calculate Realized Volatility
df['realized_volatility'] = df['Return'].rolling(window=21).std() * np.sqrt(252)

In [ ]:
df = df.dropna()

In [ ]:
# Scale Teturn for GARCH
garch_return = df['Return'] * 100

#Defire and fit the model
model_garch = arch_model(garch_return, vol='Garch', p=1, q=1)
model_garch_fit = model_garch.fit(update_freq=5)

#get the forecasted volatility
df ['garch_forecast'] = model_garch_fit.conditional_volatility / 100 * np.sqrt(252)

Iteration:      5,   Func. Count:     37,   Neg. LLF: 7019.754430100437
Iteration:     10,   Func. Count:     63,   Neg. LLF: 6696.756355784778
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6696.755383953927
            Iterations: 14
            Function evaluations: 82
            Gradient evaluations: 14


In [ ]:
#Create data for LTSM
data_for_ltsm = df['realized_volatility'].values
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_for_ltsm.reshape(-1,1))

In [ ]:
#Create sequences
lookback = 30
X, y = [], []
for i in range(lookback, len(data_scaled)):
    X.append(data_scaled[i-lookback:i, 0])
    y.append(data_scaled[i, 0])

X, y = np.array(X), np.array(y)

#Reshape for LTSM
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [ ]:
#Train & Test split

split_index = int(len(X) * 0.8)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [ ]:
#Build and fit the model

model_ltsm = Sequential()
model_ltsm.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_ltsm.add(Dropout(0.2))
model_ltsm.add(LSTM(units=50))
model_ltsm.add(Dropout(0.2))
model_ltsm.add(Dense(units=1))

model_ltsm.compile(optimizer='adam', loss='mean_squared_error')
model_ltsm.fit(X_train, y_train, epochs=50, batch_size=32)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.0131
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0024
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0017
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0019
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0016
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0015
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0013
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0012
Epoch 9/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0013
Epoch 10/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0011
Epoch 11/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0013
Epoch 12/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0011
Epoch 13/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0010
Epoch 14/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 9.6334e-04
Epoch 15/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0011
Epoch 16/50
87

In [ ]:
#LTSM Predictions
predictions_ltsm = model_ltsm.predict(X_test)
predictions_ltsm = scaler.inverse_transform(predictions_ltsm)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

#Garch predictions for the same period
garch_test_forecast = df['garch_forecast'].iloc[split_index + lookback:]

#calculate Error (RMSE)
rmse_ltsm = np.sqrt(mean_squared_error(y_test, predictions_ltsm))
rmse_garch = np.sqrt(mean_squared_error(y_test, garch_test_forecast.values))

print(f"LTSM RMSE: {rmse_ltsm}")
print(f"GARCH RMSE: {rmse_garch}")

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
LTSM RMSE: 0.016099092630026925
GARCH RMSE: 0.04448077842974344
